In [1]:
# Room 1: Engine Room

tool_kit = {"name": "tool kit", "type": "equipment"}
locker = {"name": "locker", "type": "equipment"}
door_a = {"name": "door a", "type": "door"}
key_a = {"name": "key for door a", "type": "key", "target": door_a,}
engine_room = {"name": "engine room", "type": "room"}

# Room 2: Control Room

drawer = {"name": "drawer", "type": "equipment"}
wood_box = {"name": "wood box", "type": "equipment"}
door_a = {"name": "door a", "type": "door"}
door_b = {"name": "door b", "type": "door"}
door_c = {"name": "door c", "type": "door"}
key_b = {"name": "key for door b", "type": "key","target": door_b,}
control_room = {"name": "control room", "type": "room"}

# Room 3: Crew Room

crew_room = {"name": "crew room", "type": "room"}
captain_bed = {"name": "captain's bed", "type": "equipment"}
sink = {"name": "sink", "type": "equipment"}
oven = {"name": "oven", "type": "equipment"}
door_b = {"name": "door b", "type": "door"}
key_c = {"name": "key for door c", "type": "key","target": door_c}

# Room 4: Upper Deck

door_d = {"name": "door d", "type": "door"}
key_d = {"name": "key for door d", "type": "key", "target": door_d}
stairs = {"name": "stairs", "type": "equipment"}
door_c = {"name": "door c", "type": "door"}
upper_deck = {"name": "upper deck", "type": "room"}

# Room Safety Boat

safety_boat = {"name": "safety boat", "type": "room"}

all_rooms = [engine_room, control_room, crew_room, engine_room, safety_boat, upper_deck]

all_doors = [door_a, door_b, door_c, door_d]

# define which items/rooms are related

object_relations = {
    "engine room": [tool_kit, locker, door_a],
    "locker": [key_a],
    "control room": [drawer, wood_box, door_a, door_b, door_c],
    "drawer": [key_b],
    "crew room": [captain_bed, sink, oven, door_b],
    "sink":[key_c],
    "upper deck": [safety_boat, stairs, door_c, door_d],
    "stairs": [key_d],
    "safety boat":[door_d],
    "door a": [engine_room, control_room],
    "door b": [control_room, crew_room],
    "door c": [control_room, upper_deck],
    "door d": [upper_deck, safety_boat]
}

INIT_GAME_STATE = {
    "current_room": engine_room,
    "keys_collected": [],
    "target_room": safety_boat
}

In [2]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print("You wake up in the engine room of a sinking ship. The room is dark, and you can hear water rushing in. You feel scared and confused. Time is running out, and you need to find a key to open the locked door and find a way to escape to the upper deck where there is safety boats before it's too late.")
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the the sinking ship!")
    else:
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine?").strip())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")
    
    if(next_room and input("Do you want to go to the next room? Ener 'yes' or 'no'").strip() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)

In [3]:
game_state = INIT_GAME_STATE.copy()

start_game()

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!
You are now in engine room
You examine locker. You find key for door a.
You are now in engine room
You examine door a. You unlock it with a key you have.
You are now in control room
You examine drawer. You find key for door b.
You are now in control room
You examine door b. You unlock it with a key you have.
You are now in control room
The item you requested is not found in the current room.
You are now in control room
You examine door b. You unlock it with a key you have.
You are now in crew room
You examine sink. You find key for door c.
You are now in crew room
You examine door b. You unlock it with a key you have.
You are now in control room
You examine door c. You unlock it with a key you have.
You are now in upper deck
Yo